In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import re
import random
from nltk import ngrams
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
import string
import re
import itertools
from tensorflow.keras import Model
from tensorflow.keras import Input
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Sequential
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
file_path = 'drive/MyDrive/Colab Notebooks/moviedata.csv'
root_path = 'drive/MyDrive/Colab Notebooks/'

In [ ]:
df = pd.read_csv(file_path)
df = df[['Rank','Title','Genre','Director','Actors']]

In [ ]:
def pre_process(text):
  text = clean_text(text)
  text = remove_spaces(text)
  return text

def remove_spaces(text):
  text = re.sub(r" ","",text)
  return text

def replace_commas(text):
  text = re.sub(r","," ",text)
  text = re.sub(r"[-{}\"#/@:;<>+=~()|.!?]","",text)
  return text

def clean_text(text):
  text = text.lower()
  text = re.sub(r"i'm", "i am", text)
  text = re.sub(r"he's", "he is", text)
  text = re.sub(r"she's", "she is", text)
  text = re.sub(r"that's", "that is", text)
  text = re.sub(r"what's", "what is", text)
  text = re.sub(r"where's", "where is", text)
  text = re.sub(r"\'ll", " will", text)
  text = re.sub(r"\'ve", " have", text)
  text = re.sub(r"\'re", " are", text)
  text = re.sub(r"\'d", " would", text)
  text = re.sub(r"won't", "will not", text)
  text = re.sub(r"can't", "cannot", text)
  text = re.sub(r"[-{}\"#/@:;<>+=~()|.?,]", "", text) 
  return text

In [ ]:
genres = set()
directors = set()
actors = set()
for index,row in df.iterrows():
  entry = row["Genre"]
  for e in entry.split(","):
    genres.add(e.lower().strip())
  entry = row["Director"]
  for e in entry.split(","):
    directors.add(e.lower().strip())
  entry = row["Actors"]
  for e in entry.split(","):
    actors.add(e.lower().strip())
print("Found %d genres"%len(genres))
print("Found %d directors"%len(directors))
print("Found %d actors"%len(actors))

Found 20 genres
Found 644 directors
Found 1985 actors


In [ ]:
data = df.copy()
# data=data.set_index(data.columns.drop('Genre',1).tolist()).Genre.str.split(',', expand=True).stack().reset_index().rename(columns={0:'Genre'}).loc[:, data.columns]
# data=data.set_index(data.columns.drop('Director',1).tolist()).Director.str.split(',', expand=True).stack().reset_index().rename(columns={0:'Director'}).loc[:, data.columns]
# data=data.set_index(data.columns.drop('Actors',1).tolist()).Actors.str.split(',', expand=True).stack().reset_index().rename(columns={0:'Actors'}).loc[:, data.columns]

In [ ]:
data['Genre'] = data['Genre'].map(lambda x: replace_commas(x)) 
data['Director'] = data['Director'].map(lambda x: replace_commas(x))
data['Actors'] = data['Actors'].map(lambda x: replace_commas(x)) 
data["Query"] = data["Genre"].str.cat(data[["Director", "Actors"]].astype(str), sep=" ")
data

,Rank,Title,Genre,Director,Actors,Query
0,1,Guardians of the Galaxy,Action Adventure SciFi,James Gunn,Chris Pratt Vin Diesel Bradley Cooper Zoe S...,Action Adventure SciFi James Gunn Chris Pratt ...
1,2,Prometheus,Adventure Mystery SciFi,Ridley Scott,Noomi Rapace Logan MarshallGreen Michael Fas...,Adventure Mystery SciFi Ridley Scott Noomi Rap...
2,3,Split,Horror Thriller,M Night Shyamalan,James McAvoy Anya TaylorJoy Haley Lu Richard...,Horror Thriller M Night Shyamalan James McAvoy...
3,4,Sing,Animation Comedy Family,Christophe Lourdelet,Matthew McConaughey Reese Witherspoon Seth Ma...,Animation Comedy Family Christophe Lourdelet M...
4,5,Suicide Squad,Action Adventure Fantasy,David Ayer,Will Smith Jared Leto Margot Robbie Viola D...,Action Adventure Fantasy David Ayer Will Smith...
...,...,...,...,...,...,...
995,996,Secret in Their Eyes,Crime Drama Mystery,Billy Ray,Chiwetel Ejiofor Nicole Kidman Julia Roberts...,Crime Drama Mystery Billy Ray Chiwetel Ejiofor...
996,997,Hostel: Part II,Horror,Eli Roth,Lauren German Heather Matarazzo Bijou Philli...,Horror Eli Roth Lauren German Heather Mataraz...
997,998,Step Up 2: The Streets,Drama Music Romance,Jon M Chu,Robert Hoffman Briana Evigan Cassie Ventura ...,Drama Music Romance Jon M Chu Robert Hoffman ...
998,999,Search Party,Adventure Comedy,Scot Armstrong,Adam Pally TJ Miller Thomas Middleditch Shan...,Adventure Comedy Scot Armstrong Adam Pally TJ...


In [ ]:
data[data.Title == 'Jurassic World']['Query']

85    Action Adventure SciFi Colin Trevorrow Chris P...
Name: Query, dtype: object

In [ ]:
count_vector = CountVectorizer()
count_matrix = count_vector.fit_transform(data["Query"])
similarity_scores = cosine_similarity(count_matrix)

In [ ]:
len(similarity_scores)

1000

In [ ]:
# data[data['Query'] == "Sci-Fi Colin Trevorrow Bryce Dallas Howard"]

,Rank,Title,Genre,Director,Actors,Query


In [ ]:
# data[data.index == movie_index]

In [ ]:
# data[data.Title == 'Aliens vs Predator - Requiem']

In [ ]:
def get_title(index,data):
    return data[data.index == index]["Title"].values[0]
def get_index(title,data):
    return data[data.Title == title].index[0]

In [ ]:
movie_user_likes = "Action Adventure SciFi Colin Trevorrow Chris"
movie_index = data[data.Query == movie_user_likes].index[0]
data[data.index == movie_index]
print(movie_index)
similar_movies = list(enumerate(similarity_scores[movie_index])) 
sorted_similar_movies = sorted(similar_movies,key=lambda x:x[1],reverse=True)[1:]
movies = set()
for entry in sorted_similar_movies:
  if entry[1] > 0.5:
    movies.add(get_title(entry[0],data))
print(movies)

1618
{'Aliens vs Predator - Requiem', 'Jurassic World'}


In [ ]:
movie_user_likes = "Furious 6"
movie_index = get_index(movie_user_likes,data)
print(movie_index,data[data.index == movie_index])
similar_movies = list(enumerate(similarity_scores[movie_index])) 
sorted_similar_movies = sorted(similar_movies,key=lambda x:x[1],reverse=True)[1:]
movies = set()
for entry in sorted_similar_movies:
  if entry[1] > 0.4:
    movies.add(get_title(entry[0],data))
print(movies)

340      Rank  ...                                              Query
340   341  ...  Action Crime Thriller Justin Lin Vin Diesel  P...

[1 rows x 6 columns]
{'Fast Five', 'Furious Seven', 'Furious 6'}


{'Furious Seven', 'Guardians of the Galaxy'}
